In [6]:
import xmltodict
import pandas as pd
import requests
import json

req = requests.get('http://knesset.gov.il/Odata/ParliamentInfo.svc/KNS_Bill()')
dickt = xmltodict.parse(req.content)
xml_entry = dickt.get("feed").get("entry")
df_cols = ["BillID", "KnessetNum", "Name", "SubTypeID","SubTypeDesc","CommitteeID", "PublicationDate"]

df = pd.DataFrame(columns=df_cols)
for i in range(len(xml_entry)):
    table = str(xml_entry[i]).replace("\"","").replace("None", "\"None\"").replace("\'", "\"")
    table = json.loads(table)
    line = table["content"]["m:properties"]
    dick = {"BillID":str(line["d:BillID"].get("#text")),
                "KnessetNum":str(line["d:KnessetNum"].get("#text")),
                "Name":str(line["d:Name"]), "SubTypeID":str(line["d:SubTypeID"].get("#text")),
                "SubTypeDesc":str(line["d:SubTypeDesc"]),"CommitteeID":str(line["d:CommitteeID"].get("#text")),
                "PublicationDate":str(line["d:PublicationDate"].get("#text"))}
    df.loc[i] = list(dick.values())
df.head(10)

,BillID,KnessetNum,Name,SubTypeID,SubTypeDesc,CommitteeID,PublicationDate
0,5,1,"חוק שכר חברי הכנסת, התשט-1949",53,ממשלתית,377,1949-06-07T00:00:00
1,20,7,"חוק מקצועות רפואיים (אגרות), התשלא-1971",53,ממשלתית,280,1971-04-07T00:00:00
2,15752,16,"חוק רשות הספנות והנמלים, התשסד-2004",53,ממשלתית,21,2004-07-22T00:00:00
3,16126,16,"חוק לתיקון פקודת המשטרה (מס"" 18), התשסד-2004",53,ממשלתית,25,2004-06-30T00:00:00
4,16127,17,"חוק להגנת חיית הבר (תיקון מס"" 6), התשסט-2008",53,ממשלתית,200,2008-11-05T00:00:00
5,16399,16,"חוק סדר הדין הפלילי (תיקון מס"" 33) (הוראת שעה)...",53,ממשלתית,25,2004-06-30T00:00:00
6,16403,16,"חוק ההוצאה לפועל (תיקון מס"" 24) (תיקון), התשסד...",53,ממשלתית,25,2003-11-12T00:00:00
7,16633,16,חוק המדיניות הכלכלית לשנת הכספים 2004 (תיקוני ...,53,ממשלתית,21,2004-01-18T00:00:00
8,16634,16,"חוק גיל פרישה, התשסד-2004",53,ממשלתית,21,2004-01-18T00:00:00
9,16639,16,"הצעת חוק נכי רדיפות הנאצים (תיקון - כפל קצבה),...",54,פרטית,21,None


In [10]:
import sqlite3

connection = sqlite3.connect('kneset')
con = connection.cursor()
con.execute('CREATE TABLE IF NOT EXISTS kneset (BillID, KnessetNum, Name, SubTypeID,SubTypeDesc,CommitteeID, PublicationDate)')
connection.commit()
df.to_sql('keneset', connection, if_exists='replace', index = False)
con.execute('''SELECT * FROM keneset''')
for line in con.fetchall():
    print(line)

('5', '1', 'חוק שכר חברי הכנסת, התשט-1949', '53', 'ממשלתית', '377', '1949-06-07T00:00:00')
('20', '7', 'חוק מקצועות רפואיים (אגרות), התשלא-1971', '53', 'ממשלתית', '280', '1971-04-07T00:00:00')
('15752', '16', 'חוק רשות הספנות והנמלים, התשסד-2004', '53', 'ממשלתית', '21', '2004-07-22T00:00:00')
('16126', '16', 'חוק לתיקון פקודת המשטרה (מס" 18), התשסד-2004', '53', 'ממשלתית', '25', '2004-06-30T00:00:00')
('16127', '17', 'חוק להגנת חיית הבר (תיקון מס" 6), התשסט-2008', '53', 'ממשלתית', '200', '2008-11-05T00:00:00')
('16399', '16', 'חוק סדר הדין הפלילי (תיקון מס" 33) (הוראת שעה) (תיקון), התשסד-2004', '53', 'ממשלתית', '25', '2004-06-30T00:00:00')
('16403', '16', 'חוק ההוצאה לפועל (תיקון מס" 24) (תיקון), התשסד-2003', '53', 'ממשלתית', '25', '2003-11-12T00:00:00')
('16633', '16', 'חוק המדיניות הכלכלית לשנת הכספים 2004 (תיקוני חקיקה), התשסד-2004', '53', 'ממשלתית', '21', '2004-01-18T00:00:00')
('16634', '16', 'חוק גיל פרישה, התשסד-2004', '53', 'ממשלתית', '21', '2004-01-18T00:00:00')
('16639', '16',